In [42]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

In [ ]:
# Load the data and libraries
import pandas as pd
import numpy as np

bear = pd.read_csv('https://raw.githubusercontent.com/jbennett979/Data_Privacy_FP/refs/heads/main/north_america_bear_killings.csv')

In [ ]:
# Load data files
import numpy as np
import urllib.request
import io

url_x_pre = pd.read_csv('https://raw.githubusercontent.com/jbennett979/Data_Privacy_FP/refs/heads/main/north_america_bear_killings_processed_x.csv')
url_y_pre = pd.read_csv('https://raw.githubusercontent.com/jbennett979/Data_Privacy_FP/refs/heads/main/north_america_bear_killings_processed_y.csv')

X = url_x_pre.to_numpy()
y = url_y_pre.to_numpy()


In [ ]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))


In [ ]:
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)

    if norm > b:
        return b * (v / norm)
    else:
        return v

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    print(xi)
    print(yi)
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.zeros(X_train.shape[1])
    noisy_count = laplace_mech(len(X_train), sensitivity=1, epsilon=epsilon)
    for i in range(iterations):
        grads = [gradient(theta, x_i, y_i) for x_i, y_i in zip(X_train, y_train)]
        b = 3 # clipping parameter (for the L2)
        clipped_grads = [L2_clip(g, b) for g in grads]
        sum_grad = np.sum(clipped_grads, axis=0)
        noisy_sum = gaussian_mech_vec(sum_grad, sensitivity=b, epsilon=epsilon, delta=delta)
        noisy_grad = np.array(noisy_sum) / noisy_count
        theta = theta - noisy_grad
    return theta

theta = noisy_gradient_descent(10, 10.0, 1e-5)
print('Final accuracy:', accuracy(theta))